In [1]:
from pathlib import Path

In [2]:
input_folder = 'Watch - TV Shows (Anime)'
input_path = Path('f:/') / input_folder # f in asus, d in lenovo

In [3]:
folder_list = [path for path in input_path.iterdir() if path.is_dir()]

In [4]:
for folder_path in folder_list:
    print(folder_path.name)
    file_list = list(folder_path.glob('*.mp4'))
    for file_path in file_list:
        print(file_path.name)
    print()

[86_Eighty-Six][不存在的战区][01-23][CHS][1080P][MP4]
86_Eighty-Six.不存在的战区.S01E01.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E02.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E03.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E04.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E05.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E06.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E07.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E08.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E09.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E10.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E11.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E12.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E13.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E14.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E15.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E16.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E17.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E18.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E19.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E20.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E21.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E22.CHS.1080P.mp4
86_Eighty-Six.不存在的战区.S01E23.